In [8]:
import requests as rq
import json
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd

In [2]:
def save_jsonl(json_list, output_file_path):
    with open(output_file_path, 'w', encoding="utf-8") as output_file:
        for sample in json_list:
            json_line = json.dumps(sample, ensure_ascii=False)
            #json.dumps(sample, output_file)
            output_file.write(json_line + '\n')

def read_jsonl(read_file_path):
    with open(read_file_path, encoding="utf-8") as f:
        data = [json.loads(i) for i in f]
    return data

In [3]:
lenta_topics = {
    0 : 'Общество/Россия',
    4 : 'Экономика',
    37 : 'Силовые структуры',
    3 : 'Бывший СССР',
    8 : 'Спорт',
    87: 'Забота о себе',
    48: 'Туризм/Путешествия',
    5 : 'Наука и техника'
}
work_path = Path('.')

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = [
            {
                'id': i.pop('docid'), 
                'url': i.pop('url'), 
                'title': i.pop('title'), 
                'subtitle': None, 
                'topic': lenta_topics[i.pop('bloc')], 
                'content': i.pop('text'), 
                'datetime': datetime.fromtimestamp(i.pop('modified')).strftime('%H:%M %d.%m.%Y')
                } for i in r.json()['matches']
            ]

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,):
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        timedelta_s = param_copy.pop('timedelta_step')
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = []
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out+= self._get_search_table(param_copy)
            dateFrom += time_step + timedelta(days=timedelta_s)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1

        save_jsonl(out, work_path.joinpath('data/lenta').joinpath(f'lenta_{lenta_topics[int(param_dict["bloc"])]}.jsonl').__str__())
        return out

In [ ]:
# Задаем тут параметры
query = 'РБК'
offset = 0
size = 100000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0" # topic = тематика новости
dateFrom = '2023-01-01'
dateTo = "2024-12-23"
timedelta_step = 3

param_dict = {
    'query'     : query,
    'timedelta_step': timedelta_step,
    'from'      : str(offset),
    'size'      : str(size),
    'dateFrom'  : dateFrom,
    'dateTo'    : dateTo,
    'sort'      : sort,
    'title_only': title_only,
    'type'      : material,
    'bloc'      : bloc,
    'domain'    : domain
    }

In [ ]:
for bloc_topic in lenta_topics:
    param_dict.update({'bloc': bloc_topic})
    parser = lentaRu_parser()
    tbl = parser.get_articles(
        param_dict=param_dict,
        time_step = 180
        )
    print(f'DONE {lenta_topics[bloc_topic]} = {len(tbl)}')